# TMDB Movie Recommendation System

# Import Dependencies

In [138]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pandarallel import pandarallel
import pickle

# Import Dataset

In [10]:
dataset=pd.read_csv("tmdb_movies_data.csv")

# EDA

In [11]:
dataset.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,06-09-2015,5562,6.5,2015,137999939.3,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015,137999939.3,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480,6.3,2015,101199955.5,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,5292,7.5,2015,183999919.0,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,04-01-2015,2947,7.3,2015,174799923.1,1.385749e+09


In [12]:
dataset.shape

(10866, 21)

In [13]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10866 non-null  int64  
 1   imdb_id               10856 non-null  object 
 2   popularity            10866 non-null  float64
 3   budget                10866 non-null  int64  
 4   revenue               10866 non-null  int64  
 5   original_title        10866 non-null  object 
 6   cast                  10790 non-null  object 
 7   homepage              2936 non-null   object 
 8   director              10822 non-null  object 
 9   tagline               8042 non-null   object 
 10  keywords              9373 non-null   object 
 11  overview              10862 non-null  object 
 12  runtime               10866 non-null  int64  
 13  genres                10843 non-null  object 
 14  production_companies  9836 non-null   object 
 15  release_date       

In [14]:
dataset.describe()

,id,popularity,budget,revenue,runtime,vote_count,vote_average,release_year,budget_adj,revenue_adj
count,10866.000000,10866.000000,1.086600e+04,1.086600e+04,10866.000000,10866.000000,10866.000000,10866.000000,1.086600e+04,1.086600e+04
mean,66064.177434,0.646441,1.462570e+07,3.982332e+07,102.070863,217.389748,5.974922,2001.322658,1.755104e+07,5.136436e+07
std,92130.136561,1.000185,3.091321e+07,1.170035e+08,31.381405,575.619058,0.935142,12.812941,3.430616e+07,1.446325e+08
min,5.000000,0.000065,0.000000e+00,0.000000e+00,0.000000,10.000000,1.500000,1960.000000,0.000000e+00,0.000000e+00
25%,10596.250000,0.207583,0.000000e+00,0.000000e+00,90.000000,17.000000,5.400000,1995.000000,0.000000e+00,0.000000e+00
50%,20669.000000,0.383856,0.000000e+00,0.000000e+00,99.000000,38.000000,6.000000,2006.000000,0.000000e+00,0.000000e+00
75%,75610.000000,0.713817,1.500000e+07,2.400000e+07,111.000000,145.750000,6.600000,2011.000000,2.085325e+07,3.369710e+07
max,417859.000000,32.985763,4.250000e+08,2.781506e+09,900.000000,9767.000000,9.200000,2015.000000,4.250000e+08,2.827124e+09


In [20]:
#checkin for null values
dataset.isna().sum()

id                         0
imdb_id                   10
popularity                 0
budget                     0
revenue                    0
original_title             0
cast                      76
homepage                7929
director                  44
tagline                 2824
keywords                1493
overview                   4
runtime                    0
genres                    23
production_companies    1030
release_date               0
vote_count                 0
vote_average               0
release_year               0
budget_adj                 0
revenue_adj                0
dtype: int64

In [18]:
#dropinng duplicate rows
dataset.drop_duplicates(inplace=True)

In [19]:
#checking for duplicate rows
dataset.duplicated().sum()

0

In [23]:
dataset['keywords']

0        monster|dna|tyrannosaurus rex|velociraptor|island
1         future|chase|post-apocalyptic|dystopia|australia
2        based on novel|revolution|dystopia|sequel|dyst...
3                    android|spaceship|jedi|space opera|3d
4                      car race|speed|revenge|suspense|car
                               ...                        
10861                             surfer|surfboard|surfing
10862                            car race|racing|formula 1
10863                             car|trolley|stealing car
10864                                                spoof
10865                  fire|gun|drive|sacrifice|flashlight
Name: keywords, Length: 10865, dtype: object

In [21]:
dataset.columns

Index(['id', 'imdb_id', 'popularity', 'budget', 'revenue', 'original_title',
       'cast', 'homepage', 'director', 'tagline', 'keywords', 'overview',
       'runtime', 'genres', 'production_companies', 'release_date',
       'vote_count', 'vote_average', 'release_year', 'budget_adj',
       'revenue_adj'],
      dtype='object')

In [37]:
#droping unwanted columns
dataset1=dataset.drop(columns=['imdb_id','popularity', 'budget', 'revenue','homepage', 'tagline','runtime','production_companies', 'release_date','vote_count', 'vote_average', 'release_year', 'budget_adj','revenue_adj'])

In [38]:
dataset1.shape

(10865, 7)

In [39]:
dataset1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10865 entries, 0 to 10865
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              10865 non-null  int64 
 1   original_title  10865 non-null  object
 2   cast            10789 non-null  object
 3   director        10821 non-null  object
 4   keywords        9372 non-null   object
 5   overview        10861 non-null  object
 6   genres          10842 non-null  object
dtypes: int64(1), object(6)
memory usage: 679.1+ KB


In [40]:
dataset1.head()

,id,original_title,cast,director,keywords,overview,genres
0,135397,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,monster|dna|tyrannosaurus rex|velociraptor|island,Twenty-two years after the events of Jurassic ...,Action|Adventure|Science Fiction|Thriller
1,76341,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,future|chase|post-apocalyptic|dystopia|australia,An apocalyptic story set in the furthest reach...,Action|Adventure|Science Fiction|Thriller
2,262500,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,based on novel|revolution|dystopia|sequel|dyst...,Beatrice Prior must confront her inner demons ...,Adventure|Science Fiction|Thriller
3,140607,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,android|spaceship|jedi|space opera|3d,Thirty years after defeating the Galactic Empi...,Action|Adventure|Science Fiction|Fantasy
4,168259,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,car race|speed|revenge|suspense|car,Deckard Shaw seeks revenge against Dominic Tor...,Action|Crime|Thriller


In [41]:
dataset1.isna().sum()

id                   0
original_title       0
cast                76
director            44
keywords          1493
overview             4
genres              23
dtype: int64

In [42]:
dataset1.dropna(inplace=True)

In [43]:
dataset1.shape

(9305, 7)

In [44]:
dataset1.isna().sum()

id                0
original_title    0
cast              0
director          0
keywords          0
overview          0
genres            0
dtype: int64

In [45]:
pandarallel.initialize()

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [50]:
#cleaning and preparing the data
dataset1['cast']=dataset1['cast'].parallel_apply(lambda x:x.replace("|",","))
dataset1['cast']=dataset1['cast'].parallel_apply(lambda x:x.replace(" ",""))
dataset1['keywords']=dataset1['keywords'].parallel_apply(lambda x:x.replace("|",","))
dataset1['keywords']=dataset1['keywords'].parallel_apply(lambda x:x.replace(" ",""))
dataset1['genres']=dataset1['genres'].parallel_apply(lambda x:x.replace("|",","))
dataset1['genres']=dataset1['genres'].parallel_apply(lambda x:x.replace(" ",""))

In [55]:
dataset1['director']=dataset1['director'].parallel_apply(lambda x:x.replace(" ",""))

In [56]:
dataset1.head()

,id,original_title,cast,director,keywords,overview,genres
0,135397,Jurassic World,"ChrisPratt,BryceDallasHoward,IrrfanKhan,Vincen...",ColinTrevorrow,"monster,dna,tyrannosaurusrex,velociraptor,island",Twenty-two years after the events of Jurassic ...,"Action,Adventure,ScienceFiction,Thriller"
1,76341,Mad Max: Fury Road,"TomHardy,CharlizeTheron,HughKeays-Byrne,Nichol...",GeorgeMiller,"future,chase,post-apocalyptic,dystopia,australia",An apocalyptic story set in the furthest reach...,"Action,Adventure,ScienceFiction,Thriller"
2,262500,Insurgent,"ShaileneWoodley,TheoJames,KateWinslet,AnselElg...",RobertSchwentke,"basedonnovel,revolution,dystopia,sequel,dystop...",Beatrice Prior must confront her inner demons ...,"Adventure,ScienceFiction,Thriller"
3,140607,Star Wars: The Force Awakens,"HarrisonFord,MarkHamill,CarrieFisher,AdamDrive...",J.J.Abrams,"android,spaceship,jedi,spaceopera,3d",Thirty years after defeating the Galactic Empi...,"Action,Adventure,ScienceFiction,Fantasy"
4,168259,Furious 7,"VinDiesel,PaulWalker,JasonStatham,MichelleRodr...",JamesWan,"carrace,speed,revenge,suspense,car",Deckard Shaw seeks revenge against Dominic Tor...,"Action,Crime,Thriller"


In [72]:
dataset1['tags']=dataset1['cast']+" "+dataset1['director']+" "+dataset1['keywords']+" "+dataset1['overview']+" "+dataset1['genres']

In [73]:
dataset1.head()

,id,original_title,cast,director,keywords,overview,genres,tags
0,135397,Jurassic World,"ChrisPratt,BryceDallasHoward,IrrfanKhan,Vincen...",ColinTrevorrow,"monster,dna,tyrannosaurusrex,velociraptor,island",Twenty-two years after the events of Jurassic ...,"Action,Adventure,ScienceFiction,Thriller","ChrisPratt,BryceDallasHoward,IrrfanKhan,Vincen..."
1,76341,Mad Max: Fury Road,"TomHardy,CharlizeTheron,HughKeays-Byrne,Nichol...",GeorgeMiller,"future,chase,post-apocalyptic,dystopia,australia",An apocalyptic story set in the furthest reach...,"Action,Adventure,ScienceFiction,Thriller","TomHardy,CharlizeTheron,HughKeays-Byrne,Nichol..."
2,262500,Insurgent,"ShaileneWoodley,TheoJames,KateWinslet,AnselElg...",RobertSchwentke,"basedonnovel,revolution,dystopia,sequel,dystop...",Beatrice Prior must confront her inner demons ...,"Adventure,ScienceFiction,Thriller","ShaileneWoodley,TheoJames,KateWinslet,AnselElg..."
3,140607,Star Wars: The Force Awakens,"HarrisonFord,MarkHamill,CarrieFisher,AdamDrive...",J.J.Abrams,"android,spaceship,jedi,spaceopera,3d",Thirty years after defeating the Galactic Empi...,"Action,Adventure,ScienceFiction,Fantasy","HarrisonFord,MarkHamill,CarrieFisher,AdamDrive..."
4,168259,Furious 7,"VinDiesel,PaulWalker,JasonStatham,MichelleRodr...",JamesWan,"carrace,speed,revenge,suspense,car",Deckard Shaw seeks revenge against Dominic Tor...,"Action,Crime,Thriller","VinDiesel,PaulWalker,JasonStatham,MichelleRodr..."


In [74]:
new_dataset=dataset1[['id','original_title','tags']]

In [75]:
new_dataset.head()

,id,original_title,tags
0,135397,Jurassic World,"ChrisPratt,BryceDallasHoward,IrrfanKhan,Vincen..."
1,76341,Mad Max: Fury Road,"TomHardy,CharlizeTheron,HughKeays-Byrne,Nichol..."
2,262500,Insurgent,"ShaileneWoodley,TheoJames,KateWinslet,AnselElg..."
3,140607,Star Wars: The Force Awakens,"HarrisonFord,MarkHamill,CarrieFisher,AdamDrive..."
4,168259,Furious 7,"VinDiesel,PaulWalker,JasonStatham,MichelleRodr..."


# Creating a PorterStemmer Variable

In [76]:
ps=PorterStemmer()

# Buliding a method to apply PortStemmer

In [77]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return y

In [78]:
new_dataset['tags']=new_dataset['tags'].apply(stem)

C:\Users\VENKTESH DALAVAYI\AppData\Local\Temp\ipykernel_3084\1283114390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataset['tags']=new_dataset['tags'].apply(stem)


In [80]:
new_dataset['tags']

0        [chrispratt,brycedallashoward,irrfankhan,vince...
1        [tomhardy,charlizetheron,hughkeays-byrne,nicho...
2        [shailenewoodley,theojames,katewinslet,anselel...
3        [harrisonford,markhamill,carriefisher,adamdriv...
4        [vindiesel,paulwalker,jasonstatham,michellerod...
                               ...                        
10861    [michaelhynson,robertaugust,lord'tallyho'blear...
10862    [jamesgarner,evamariesaint,yvesmontand,toshirå...
10863    [innokentiysmoktunovskiy,olegefremov,georgizhz...
10864    [tatsuyamihashi,akikowakabayashi,miehama,johns...
10865    [haroldp.warren,tomneyman,johnreynolds,dianema...
Name: tags, Length: 9305, dtype: object

In [82]:
new_dataset['tags']=new_dataset['tags'].parallel_apply(lambda x:" ".join(x))

C:\Users\VENKTESH DALAVAYI\AppData\Local\Temp\ipykernel_3084\616289913.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataset['tags']=new_dataset['tags'].parallel_apply(lambda x:" ".join(x))


In [115]:
new_dataset['tags']=new_dataset['tags'].parallel_apply(lambda x:x.lower())

C:\Users\VENKTESH DALAVAYI\AppData\Local\Temp\ipykernel_3084\2395327489.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataset['tags']=new_dataset['tags'].parallel_apply(lambda x:x.lower())


In [116]:
new_dataset.shape

(9305, 3)

In [117]:
new_dataset.reset_index(drop=True,inplace=True)

In [118]:
new_dataset

,id,original_title,tags
0,135397,Jurassic World,"chrispratt,brycedallashoward,irrfankhan,vincen..."
1,76341,Mad Max: Fury Road,"tomhardy,charlizetheron,hughkeays-byrne,nichol..."
2,262500,Insurgent,"shailenewoodley,theojames,katewinslet,anselelg..."
3,140607,Star Wars: The Force Awakens,"harrisonford,markhamill,carriefisher,adamdrive..."
4,168259,Furious 7,"vindiesel,paulwalker,jasonstatham,michellerodr..."
...,...,...,...
9300,21,The Endless Summer,"michaelhynson,robertaugust,lord'tallyho'blears..."
9301,20379,Grand Prix,"jamesgarner,evamariesaint,yvesmontand,toshiråm..."
9302,39768,Beregis Avtomobilya,"innokentiysmoktunovskiy,olegefremov,georgizhzh..."
9303,21449,"What's Up, Tiger Lily?","tatsuyamihashi,akikowakabayashi,miehama,johnse..."


# Creating a CountVectorizer Variable

In [119]:
cv=CountVectorizer(max_features=9350,stop_words='english')

In [120]:
vectors=cv.fit_transform(new_dataset['tags']).toarray()

In [121]:
vectors.shape

(9305, 9350)

In [122]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

# Creating a cosine similarity variable

In [123]:
similarity=cosine_similarity(vectors)

In [124]:
similarity.shape

(9305, 9305)

In [125]:
similarity[0]

array([1.        , 0.14517324, 0.1204829 , ..., 0.        , 0.06558258,
       0.        ])

In [126]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda vector:vector[1])[1:5]

[(4396, 0.3629330931556489),
 (8697, 0.3486948369707025),
 (8668, 0.31773541587959875),
 (5702, 0.21251185925162067)]

# Building a method to recommend movies

In [134]:
def recommend(movies):
    movie_index=new_dataset[new_dataset['original_title']==movies].index[0]
    distance=similarity[movie_index]
    movies_list=sorted(list(enumerate(distance)),reverse=True,key=lambda vector:vector[1])[1:11]
    for i in movies_list:
        print(new_dataset.iloc[i[0]].original_title)

In [135]:
recommend("Jurassic World")

The Lost World: Jurassic Park
Jurassic Park
Futureworld
Dark Ride
Universal Soldier: Regeneration
Shadow Man
Farscape The Peacekeeper Wars
Marvel One-Shot: Agent Carter
Bionicle: Mask of Light
The Good Dinosaur


In [136]:
recommend("Mad Max: Fury Road")

Steel Dawn
Mad Max
Hammer of the Gods
The Book of Eli
Comanche Station
The 6th Day
The Maze Runner
9
Black Dynamite
Big Jake


In [137]:
recommend("Mad Max")

Torque
Turbo Kid
Steel Dawn
Mad Max 2
The Book of Eli
Mad Max: Fury Road
Banlieue 13
Homefront
Judge Dredd
Resident Evil: Extinction


# Saving the new dataset and cosine similarity

In [140]:
pickle.dump(new_dataset,open("tmdb_new_dataset.pkl","wb"))
pickle.dump(similarity,open("similarity.pkl","wb"))